# Feature transformation

ML algorithms typically require numerical and ordered features. Numerical means that the type of the variable must be an integer or a real number. Ordered means that the order of the values of a variable is meaningful.

In this dataset, the only numerical and ordered features are the transaction amount and the fraud label. The date is a panda timestamp, and therefore not numerical. The identifiers for the transactions, customers and terminals are numerical but not ordered: it wouldn't make any sense to assume for example that the terminal with ID 3548 is larger than the terminal with the ID 1983. Rather these identifiers represent distinct 'entities' which are referred to as categorical features

The goal of feature engineering is to design new features that are assumed to be relevant for a predictive problem. The design of these features is usually problem dependent and involves domain knowledge.

## Imports

In [36]:
# General
import os
import pandas as pd
import numpy as np
import datetime

## Helper functions

In [22]:
# Load a set of pickle files, put them together in a single DataFrame, and order them by time
# It takes as input the folder DIR_INPUT where the files are stored, and the BEGIN_DATE and END_DATE
def read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE):
    
    files = [os.path.join(DIR_INPUT, f) for f in os.listdir(DIR_INPUT) if f>=BEGIN_DATE+'.pkl' and f<=END_DATE+'.pkl']

    frames = []
    for f in files:
        df = pd.read_pickle(f)
        frames.append(df)
        del df
    df_final = pd.concat(frames)
    
    df_final=df_final.sort_values('TRANSACTION_ID')
    df_final.reset_index(drop=True,inplace=True)
    #  Note: -1 are missing values for real world data 
    df_final=df_final.replace([-1],0)
    
    return df_final

def is_weekend(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    # Binary value: 0 if weekday, 1 if weekend
    is_weekend = weekday>=5
    
    return int(is_weekend)

def is_night(tx_datetime):
    
    # Get the hour of the transaction
    tx_hour = tx_datetime.hour
    # Binary value: 1 if hour less than 6, and 0 otherwise
    is_night = tx_hour<=6
    
    return int(is_night)

def get_customer_spending_behaviour_features(customer_tx, window_size_in_days=[1,7,30]):
    # Order transactions chronologically
    customer_tx = customer_tx.sort_values('TX_DATETIME')
    
    # TX_DATETIME set as the index to allow the use of the rolling function
    customer_tx.index = customer_tx.TX_DATETIME
    
    for window_size in window_size_in_days:
        sum_tx_amount = customer_tx['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        nb_tx = customer_tx['TX_AMOUNT'].rolling(str(window_size)+'d').count()
        
        avg_tx_amount = sum_tx_amount/nb_tx
        
        customer_tx['CUSTOMER_ID_NB_TX_'+str(window_size)+'DAY_WINDOW'] = list(nb_tx)
        customer_tx['CUSTOMER_ID_AVG_AMOUNT_'+str(window_size)+'DAY_WINDOW'] = list(avg_tx_amount)
    
    # Reindex according to transaction IDs
    customer_tx.index=customer_tx.TRANSACTION_ID
    
    # Return dataframe with the new features
    return customer_tx
    
def calculate_risk_scores(terminal_tx, delay_period=7, window_size_in_days=[1,7,30], feature="TERMINAL_ID"):
    
    terminal_tx = terminal_tx.sort_values('TX_DATETIME')
    terminal_tx.index = terminal_tx.TX_DATETIME
    
    NB_FRAUD_DELAY = terminal_tx['TX_FRAUD'].rolling(str(delay_period) + 'd').sum()
    NB_TX_DELAY = terminal_tx['TX_FRAUD'].rolling(str(delay_period) + 'd').count()
    
    for window_size in window_size_in_days:
        NB_FRAUD_DELAY_WINDOW = terminal_tx['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW = terminal_tx['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
        
        NB_FRAUD_WINDOW = NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW = NB_TX_DELAY_WINDOW-NB_TX_DELAY
        
        RISK_WINDOW = NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_tx[feature+'_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_tx[feature+'_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_tx.index=terminal_tx.TRANSACTION_ID
    
    # Replace NA values with 0 (all undefined risk scores where NB_TX_WINDOW is 0)
    terminal_tx.fillna(0,inplace=True)
    return terminal_tx
    

## Load Dataset

In [23]:
# Load data from the 2021-01-01 to the 2021-07-02

DIR_INPUT='./simulated-data-raw/' 

BEGIN_DATE = "2021-01-01"
END_DATE = "2021-07-02"

print("Load  files")
%time df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(df),df.TX_FRAUD.sum()))

Load  files
CPU times: user 3.31 s, sys: 27.9 ms, total: 3.34 s
Wall time: 3.35 s
1754155 transactions loaded, containing 14681 fraudulent transactions


Features that characterise a payment card transaction:  
- Unique identifier for the transaction
- Date and time of the transaction
- Transaction amount
- Unique identifier for the customer
- Unique number for the merchant
- Binary target variable

In [24]:
df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2021-01-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2021-01-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2021-01-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2021-01-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2021-01-01 00:10:34,927,9906,50.99,634,0,0,0


### 1. Date and Time transformations

Create two new binary features from the transaction dates and times:  
    - `TX_DURING_WEEKEND` - 0 (weekday) 1 (weekend)  
    - `TX_DURING_NIGHT` - 0 (day) 1 (night)

In [25]:
%time df['TX_DURING_WEEKEND']=df.TX_DATETIME.apply(is_weekend)

CPU times: user 1.43 s, sys: 26.9 ms, total: 1.45 s
Wall time: 1.46 s


In [26]:
%time df['TX_DURING_NIGHT']=df.TX_DATETIME.apply(is_night)

CPU times: user 1.27 s, sys: 34.9 ms, total: 1.31 s
Wall time: 1.3 s


Check that these features were correctly computed

In [27]:
df[df.TX_TIME_DAYS>=30]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT
288062,288062,2021-01-31 00:01:21,3546,2944,18.71,2592081,30,0,0,1,1
288063,288063,2021-01-31 00:01:48,206,3521,18.60,2592108,30,0,0,1,1
288064,288064,2021-01-31 00:02:22,2610,4470,66.67,2592142,30,0,0,1,1
288065,288065,2021-01-31 00:03:15,4578,1520,79.41,2592195,30,0,0,1,1
288066,288066,2021-01-31 00:03:51,1246,7809,52.08,2592231,30,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,2021-07-02 23:56:36,161,655,54.24,15810996,182,0,0,0,0
1754151,1754151,2021-07-02 23:57:38,4342,6181,1.23,15811058,182,0,0,0,0
1754152,1754152,2021-07-02 23:58:21,618,1502,6.62,15811101,182,0,0,0,0
1754153,1754153,2021-07-02 23:59:52,4056,3067,55.40,15811192,182,0,0,0,0


### 2. Customer ID Transactions

Create these features over three time windows i.e [1, 7, 30] days  
Frequency - number of transactions that occur within a time window  
Monetary value - average amount spent in these transactions

In [28]:
%time df = df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, window_size_in_days=[1,7,30]))
df=df.sort_values('TX_DATETIME').reset_index(drop=True)

CPU times: user 10.1 s, sys: 14.6 ms, total: 10.1 s
Wall time: 10.2 s


In [29]:
df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
0,0,2021-01-01 00:00:31,596,3156,57.16,31,0,0,0,0,1,1.0,57.16,1.0,57.16,1.0,57.16
1,1,2021-01-01 00:02:10,4961,3412,81.51,130,0,0,0,0,1,1.0,81.51,1.0,81.51,1.0,81.51
2,2,2021-01-01 00:07:56,2,1365,146.00,476,0,0,0,0,1,1.0,146.00,1.0,146.00,1.0,146.00
3,3,2021-01-01 00:09:29,4128,8737,64.49,569,0,0,0,0,1,1.0,64.49,1.0,64.49,1.0,64.49
4,4,2021-01-01 00:10:34,927,9906,50.99,634,0,0,0,0,1,1.0,50.99,1.0,50.99,1.0,50.99


### 3. Terminal ID transformations

Main goal will be to extract a risk score that assesses the exposure of a given terminal ID to fraudulent transactions.  
The risk score will be defined as the average number of fraudulent transactions that occurred on a terminal ID over a time window. 

Contrary to customer ID transformations, the time windows will not directly precede a given transaction. Instead, will be shifted back by a delay period. The delay period accounts for the fact that, in practice, fraudulent transactions are only discovered after a fraudulent investigation or a customer complaint. Hence, the fraudulent labels, which are needed to compute the risk score, are only available after this delay period.

These features will be created over three time windows i.e [1,7,30] days


In [33]:
%time df=df.groupby('TERMINAL_ID').apply(lambda x: calculate_risk_scores(x, delay_period=7, window_size_in_days=[1,7,30], feature="TERMINAL_ID"))
df=df.sort_values('TX_DATETIME').reset_index(drop=True)

CPU times: user 23.8 s, sys: 137 ms, total: 23.9 s
Wall time: 24 s


In [34]:
df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2021-01-01 00:00:31,596,3156,57.16,31,0,0,0,0,...,1.0,57.160000,1.0,57.160000,0.0,0.0,0.0,0.0,0.0,0.00000
1,1,2021-01-01 00:02:10,4961,3412,81.51,130,0,0,0,0,...,1.0,81.510000,1.0,81.510000,0.0,0.0,0.0,0.0,0.0,0.00000
2,2,2021-01-01 00:07:56,2,1365,146.00,476,0,0,0,0,...,1.0,146.000000,1.0,146.000000,0.0,0.0,0.0,0.0,0.0,0.00000
3,3,2021-01-01 00:09:29,4128,8737,64.49,569,0,0,0,0,...,1.0,64.490000,1.0,64.490000,0.0,0.0,0.0,0.0,0.0,0.00000
4,4,2021-01-01 00:10:34,927,9906,50.99,634,0,0,0,0,...,1.0,50.990000,1.0,50.990000,0.0,0.0,0.0,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,2021-07-02 23:56:36,161,655,54.24,15810996,182,0,0,0,...,12.0,67.047500,72.0,69.521111,1.0,0.0,4.0,0.0,28.0,0.00000
1754151,1754151,2021-07-02 23:57:38,4342,6181,1.23,15811058,182,0,0,0,...,21.0,22.173810,93.0,24.780753,1.0,0.0,9.0,0.0,39.0,0.00000
1754152,1754152,2021-07-02 23:58:21,618,1502,6.62,15811101,182,0,0,0,...,21.0,7.400476,65.0,7.864462,1.0,0.0,5.0,0.0,33.0,0.00000
1754153,1754153,2021-07-02 23:59:52,4056,3067,55.40,15811192,182,0,0,0,...,16.0,107.052500,51.0,102.919608,1.0,0.0,6.0,0.0,28.0,0.00000


## Saving the dataset

In [38]:
DIR_OUTPUT = "./simulated-data-transformed/"

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

for day in range(df.TX_TIME_DAYS.max()+1):
    transactions_day = df[df.TX_TIME_DAYS==day].sort_values('TX_TIME_SECONDS')
    date = start_date + datetime.timedelta(days=day)
    filename_output = date.strftime("%Y-%m-%d")+'.pkl'
    
    transactions_day.to_pickle(DIR_OUTPUT+filename_output, protocol=4)
    